Ejecutado desde VERTEX AI Notebooks Colab Enterprise

In [2]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import json
import os
import tempfile
from google.cloud import storage

Configuramos paths

In [6]:
bucket_name = "json-bucket-datos"
input_folder_name = "Yelp"
output_folder_name = "Yelp/ETL"

# Crear cliente de Google Cloud Storage
storage_client = storage.Client()

# Construir la referencia al archivo de origen en la carpeta especificada
bucket = storage_client.get_bucket(bucket_name)

input_file_name = "user.parquet"
output_file_name = "user.parquet"
review_file_name = "review.parquet"

# Construir la referencia al archivo de origen en la carpeta especificada
read_blob_name = os.path.join(output_folder_name, review_file_name)
read_input_blob = bucket.blob(read_blob_name)
blob_name = os.path.join(input_folder_name, input_file_name)
input_blob = bucket.blob(blob_name)

# Crear directorio temporal para trabajar con los archivos
temp_dir = tempfile.mkdtemp()

# Asignar el archivo de origen al directorio temporal
input_file_path = os.path.join(temp_dir, input_file_name)
input_blob.download_to_filename(input_file_path)
# Asignar el archivo de origen al directorio temporal
read_input_file_path = os.path.join(temp_dir, review_file_name)
read_input_blob.download_to_filename(read_input_file_path)

# Crear referencia al archivo de salida de Parquet
output_blob_name = os.path.join(output_folder_name, output_file_name)
output_blob = bucket.blob(output_blob_name)

# Vamos a guardar los datos transformados en un nuevo archivo de Parquet
output_file_path = os.path.join(temp_dir, output_file_name)

Leemos el archivos reviews para obtener los ids de los usuarios

In [7]:
df_reviews = pd.read_parquet(read_input_file_path)

In [8]:
yelp_users_ids = df_reviews['user_id'].tolist()

In [ ]:
Abrimos el achivo de usuarios

In [9]:
df_users = pd.read_parquet(input_file_path)

In [10]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2105597 entries, 0 to 2105596
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   name                object 
 2   review_count        int64  
 3   yelping_since       object 
 4   useful              int64  
 5   funny               int64  
 6   cool                int64  
 7   elite               object 
 8   friends             object 
 9   fans                int64  
 10  average_stars       float64
 11  compliment_hot      int64  
 12  compliment_more     int64  
 13  compliment_profile  int64  
 14  compliment_cute     int64  
 15  compliment_list     int64  
 16  compliment_note     int64  
 17  compliment_plain    int64  
 18  compliment_cool     int64  
 19  compliment_funny    int64  
 20  compliment_writer   int64  
 21  compliment_photos   int64  
dtypes: float64(1), int64(16), object(5)
memory usage: 353.4+ MB


Leemos el archivos reviews para obtener los ids de los usuarios

In [11]:
df_users = df_users[df_users['user_id'].isin(yelp_users_ids)]

In [12]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28166 entries, 0 to 2105467
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             28166 non-null  object 
 1   name                28166 non-null  object 
 2   review_count        28166 non-null  int64  
 3   yelping_since       28166 non-null  object 
 4   useful              28166 non-null  int64  
 5   funny               28166 non-null  int64  
 6   cool                28166 non-null  int64  
 7   elite               28166 non-null  object 
 8   friends             28166 non-null  object 
 9   fans                28166 non-null  int64  
 10  average_stars       28166 non-null  float64
 11  compliment_hot      28166 non-null  int64  
 12  compliment_more     28166 non-null  int64  
 13  compliment_profile  28166 non-null  int64  
 14  compliment_cute     28166 non-null  int64  
 15  compliment_list     28166 non-null  int64  
 16  co

Filtramos Columnas

In [13]:
df_users = df_users[['user_id', 'review_count', 'friends', 'fans', 'average_stars']]


Reseteamos el Índice

In [14]:
df_users.reset_index(drop=True, inplace=True)

Guardamos el archivo limpio

In [ ]:
# Guardar el DataFrame como archivo Parquet
table = pa.Table.from_pandas(df_users)
pq.write_table(table, output_file_path)

# Subir el archivo resultante Parquet a la carpeta
output_blob.upload_from_filename(output_file_path)

# Limpieza del directorio temporal
# Limpieza del directorio temporal
os.remove(input_file_path)
os.remove(output_file_path)
os.rmdir(temp_dir)